In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 24, 8
%matplotlib inline

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.utils import resample

In [2]:
def gini(solution, submission):
    df = zip(solution, submission, range(len(solution)))
    df = sorted(df, key=lambda x: (x[1],-x[2]), reverse=True)
    rand = [float(i+1)/float(len(df)) for i in range(len(df))]
    totalPos = float(sum([x[0] for x in df]))
    cumPosFound = [df[0][0]]
    for i in range(1,len(df)):
        cumPosFound.append(cumPosFound[len(cumPosFound)-1] + df[i][0])
    Lorentz = [float(x)/totalPos for x in cumPosFound]
    Gini = [Lorentz[i]-rand[i] for i in range(len(df))]
    return sum(Gini)

def normalized_gini(solution, submission):
    normalized_gini = gini(solution, submission)/gini(solution, solution)
    return normalized_gini

In [3]:
# def prepare_data():
#     non_imp = ['ps_ind_12_bin','ps_ind_13_bin','ps_ind_18_bin','ps_car_10_cat','ps_ind_11_bin','ps_ind_10_bin','ps_ind_14']
    
#     train = (pd.read_csv('../data/train.csv', na_values=-1)
#               .fillna(value=-1))
#     unwanted = list((set(train.columns[train.columns.str.startswith('ps_calc_')])|set(non_imp)))
#     train.drop(unwanted, axis=1, inplace=True)
            
    
# #     minority_class = train.target.sum()
# #     majority_class = train.target.shape[0] - minority_class

# #     df_majority = train[train['target']==0]
# #     df_minority = train[train['target']==1]

# #     df_majority_downsampled = resample(df_majority, 
# #                                      replace=False,     # sample with replacement
# #                                      n_samples=minority_class,    # to match majority class
# #                                      random_state=np.random.randint(1999)) # reproducible results

# #     train = pd.concat([df_majority_downsampled, df_minority])

    
#     test  = (pd.read_csv('../data/test.csv', na_values=-1)
#               .fillna(value=-1)
#               .drop(unwanted, axis=1)) 
    
#     y = train.target.values        
#     train = train.drop(['id', 'target'], axis=1)
    
#     test_id = test.id.values
#     test = test.drop('id', axis=1)
    
#     fl = train.columns.tolist()
    
#     return train.values, y, test.values, test_id, fl

# X, y, X_test, test_id, _ = prepare_data()

In [4]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, shuffle=True,  random_state=np.random.randint(1000), stratify=y)

In [5]:
#class_weight = {1 : (len(y_train) - sum(y_train))/y_train.shape[0], 0: sum(y_train)/y_train.shape[0]}

In [6]:
train_df = pd.read_csv('../data/train2.csv', na_values="999") # .iloc[0:200,:]
test_df = pd.read_csv('../data/test2.csv', na_values="999")

X = train_df
X = X.drop(['target'], axis=1)
y = train_df['target']

X = X.fillna(999)
test_df = test_df.fillna(999)
test_df = test_df.drop(['id'], axis=1)

X = X.values
y = y.values

In [13]:
clf = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample')

In [14]:
# A list of dictionaries to specify the parameters we want to tune.
param_grid = { "n_estimators" : [53,100,200,300,400],
               'criterion': ['gini'],
               'max_features': [2,3,4,5,6,7],
               "max_depth" : [25,30,35,50],
               'min_samples_split': [.1,.13,0.25,0.3,0.45],
               'max_leaf_nodes': [8,10,12,14,15,16],
               "min_samples_leaf" : [.1,.22,.11,.13,.25.37,.42]
               }

n_iter_search = 500
gini_scorer = make_scorer(normalized_gini, greater_is_better = True)
random_search = RandomizedSearchCV(clf, 
                                   param_distributions=param_grid,
                                   scoring = gini_scorer,
                                   n_iter=n_iter_search,
                                   n_jobs=-1,
                                   cv= StratifiedKFold(n_splits=5, shuffle=True),
                                   verbose=2)
random_search.fit(X, y)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV] n_estimators=55, min_samples_split=0.16, min_samples_leaf=0.22, max_leaf_nodes=19, max_features=2, max_depth=21, criterion=gini 
[CV] n_estimators=55, min_samples_split=0.16, min_samples_leaf=0.22, max_leaf_nodes=19, max_features=2, max_depth=21, criterion=gini 
[CV] n_estimators=55, min_samples_split=0.16, min_samples_leaf=0.22, max_leaf_nodes=19, max_features=2, max_depth=21, criterion=gini 
[CV] n_estimators=55, min_samples_split=0.16, min_samples_leaf=0.22, max_leaf_nodes=19, max_features=2, max_depth=21, criterion=gini 
[CV] n_estimators=55, min_samples_split=0.16, min_samples_leaf=0.22, max_leaf_nodes=19, max_features=2, max_depth=21, criterion=gini 
[CV] n_estimators=53, min_samples_split=0.11, min_samples_leaf=0.11, max_leaf_nodes=20, max_features=5, max_depth=25, criterion=gini 
[CV] n_estimators=53, min_samples_split=0.11, min_samples_leaf=0.11, max_leaf_nodes=20, max_features=5, max_depth=25, criterion=gini

[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   58.4s


[CV] n_estimators=50, min_samples_split=0.13, min_samples_leaf=0.1, max_leaf_nodes=19, max_features=6, max_depth=19, criterion=gini 
[CV]  n_estimators=50, min_samples_split=0.1, min_samples_leaf=0.11, max_leaf_nodes=18, max_features=2, max_depth=19, criterion=gini, total=  20.4s
[CV]  n_estimators=50, min_samples_split=0.1, min_samples_leaf=0.11, max_leaf_nodes=18, max_features=2, max_depth=19, criterion=gini, total=  19.4s
[CV] n_estimators=50, min_samples_split=0.13, min_samples_leaf=0.1, max_leaf_nodes=19, max_features=6, max_depth=19, criterion=gini 
[CV]  n_estimators=50, min_samples_split=0.1, min_samples_leaf=0.11, max_leaf_nodes=18, max_features=2, max_depth=19, criterion=gini, total=  20.5s
[CV] n_estimators=50, min_samples_split=0.1, min_samples_leaf=0.11, max_leaf_nodes=20, max_features=3, max_depth=21, criterion=gini 
[CV] n_estimators=50, min_samples_split=0.1, min_samples_leaf=0.11, max_leaf_nodes=20, max_features=3, max_depth=21, criterion=gini 
[CV]  n_estimators=53, m

[CV]  n_estimators=100, min_samples_split=0.2, min_samples_leaf=0.11, max_leaf_nodes=20, max_features=7, max_depth=21, criterion=gini, total=  43.2s
[CV] n_estimators=55, min_samples_split=0.16, min_samples_leaf=0.1, max_leaf_nodes=17, max_features=5, max_depth=18, criterion=gini 
[CV]  n_estimators=100, min_samples_split=0.2, min_samples_leaf=0.11, max_leaf_nodes=20, max_features=7, max_depth=21, criterion=gini, total=  44.6s
[CV] n_estimators=55, min_samples_split=0.16, min_samples_leaf=0.1, max_leaf_nodes=17, max_features=5, max_depth=18, criterion=gini 
[CV]  n_estimators=53, min_samples_split=0.15, min_samples_leaf=0.13, max_leaf_nodes=20, max_features=5, max_depth=23, criterion=gini, total=  16.9s
[CV] n_estimators=55, min_samples_split=0.15, min_samples_leaf=0.1, max_leaf_nodes=19, max_features=4, max_depth=18, criterion=gini 
[CV]  n_estimators=53, min_samples_split=0.15, min_samples_leaf=0.13, max_leaf_nodes=20, max_features=5, max_depth=23, criterion=gini, total=  20.4s
[CV] 

[CV] n_estimators=55, min_samples_split=0.16, min_samples_leaf=0.13, max_leaf_nodes=16, max_features=3, max_depth=25, criterion=gini 
[CV]  n_estimators=53, min_samples_split=0.16, min_samples_leaf=0.22, max_leaf_nodes=22, max_features=5, max_depth=20, criterion=gini, total=  22.0s
[CV] n_estimators=55, min_samples_split=0.16, min_samples_leaf=0.13, max_leaf_nodes=16, max_features=3, max_depth=25, criterion=gini 
[CV]  n_estimators=53, min_samples_split=0.16, min_samples_leaf=0.22, max_leaf_nodes=22, max_features=5, max_depth=20, criterion=gini, total=  21.7s
[CV] n_estimators=55, min_samples_split=0.16, min_samples_leaf=0.13, max_leaf_nodes=16, max_features=3, max_depth=25, criterion=gini 
[CV]  n_estimators=53, min_samples_split=0.16, min_samples_leaf=0.22, max_leaf_nodes=22, max_features=5, max_depth=20, criterion=gini, total=  21.4s
[CV] n_estimators=60, min_samples_split=0.1, min_samples_leaf=0.11, max_leaf_nodes=16, max_features=4, max_depth=21, criterion=gini 
[CV]  n_estimators

...

[CV]  n_estimators=52, min_samples_split=0.22, min_samples_leaf=0.15, max_leaf_nodes=21, max_features=6, max_depth=20, criterion=gini, total=  19.6s
[CV] n_estimators=52, min_samples_split=0.16, min_samples_leaf=0.11, max_leaf_nodes=16, max_features=3, max_depth=25, criterion=gini 
[CV]  n_estimators=52, min_samples_split=0.22, min_samples_leaf=0.15, max_leaf_nodes=21, max_features=6, max_depth=20, criterion=gini, total=  19.0s
[CV] n_estimators=52, min_samples_split=0.16, min_samples_leaf=0.11, max_leaf_nodes=16, max_features=3, max_depth=25, criterion=gini 
[CV]  n_estimators=52, min_samples_split=0.22, min_samples_leaf=0.15, max_leaf_nodes=21, max_features=6, max_depth=20, criterion=gini, total=  17.4s
[CV] n_estimators=52, min_samples_split=0.16, min_samples_leaf=0.11, max_leaf_nodes=16, max_features=3, max_depth=25, criterion=gini 
[CV]  n_estimators=52, min_samples_split=0.22, min_samples_leaf=0.15, max_leaf_nodes=21, max_features=6, max_depth=20, criterion=gini, total=  18.3s
[C

[CV]  n_estimators=50, min_samples_split=0.22, min_samples_leaf=0.11, max_leaf_nodes=20, max_features=4, max_depth=17, criterion=gini, total=  21.2s
[CV] n_estimators=60, min_samples_split=0.11, min_samples_leaf=0.11, max_leaf_nodes=20, max_features=2, max_depth=18, criterion=gini 
[CV]  n_estimators=60, min_samples_split=0.2, min_samples_leaf=0.1, max_leaf_nodes=18, max_features=4, max_depth=23, criterion=gini, total=  25.2s
[CV] n_estimators=60, min_samples_split=0.11, min_samples_leaf=0.11, max_leaf_nodes=20, max_features=2, max_depth=18, criterion=gini 
[CV]  n_estimators=60, min_samples_split=0.2, min_samples_leaf=0.1, max_leaf_nodes=18, max_features=4, max_depth=23, criterion=gini, total=  24.2s
[CV] n_estimators=60, min_samples_split=0.11, min_samples_leaf=0.11, max_leaf_nodes=20, max_features=2, max_depth=18, criterion=gini 
[CV]  n_estimators=60, min_samples_split=0.2, min_samples_leaf=0.1, max_leaf_nodes=18, max_features=4, max_depth=23, criterion=gini, total=  25.4s
[CV] n_e

[CV] n_estimators=55, min_samples_split=0.2, min_samples_leaf=0.11, max_leaf_nodes=21, max_features=3, max_depth=19, criterion=gini 
[CV]  n_estimators=53, min_samples_split=0.11, min_samples_leaf=0.11, max_leaf_nodes=21, max_features=3, max_depth=25, criterion=gini, total=  22.8s
[CV]  n_estimators=100, min_samples_split=0.11, min_samples_leaf=0.11, max_leaf_nodes=18, max_features=3, max_depth=21, criterion=gini, total=  31.5s
[CV]  n_estimators=100, min_samples_split=0.11, min_samples_leaf=0.11, max_leaf_nodes=18, max_features=3, max_depth=21, criterion=gini, total=  32.1s
[CV]  n_estimators=53, min_samples_split=0.11, min_samples_leaf=0.11, max_leaf_nodes=21, max_features=3, max_depth=25, criterion=gini, total=  15.8s
[CV]  n_estimators=53, min_samples_split=0.11, min_samples_leaf=0.11, max_leaf_nodes=21, max_features=3, max_depth=25, criterion=gini, total=  15.5s
[CV]  n_estimators=55, min_samples_split=0.2, min_samples_leaf=0.11, max_leaf_nodes=21, max_features=3, max_depth=19, cr

[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed: 121.2min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
          error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=500, n_jobs=-1,
          param_distributions={'n_estimators': [50, 52, 53, 54, 55, 60, 100], 'criterion': ['gini'], 'max_features': [2, 3, 4, 5, 6, 7], 'max_depth': [16, 17, 18, 19, 20, 21, 22, 23, 25], 'min_samples_split': [0.1, 0.11, 0.12, 0.13, 0.15, 0.16, 0.2, 0.22], 'max_leaf_nodes': [16, 17, 18, 19, 20, 21, 22], 'min_samples_leaf': [0.1, 0.22, 0.11, 0.13, 0.11, 0.15]},
   

In [15]:
random_search.best_score_

0.15421225025033736

In [27]:
random_search.best_estimator_.predict_proba(test_df.values)[:,1].reshape(-1,1).shape

(892816, 1)

In [28]:
random_search.best_params_

{'criterion': 'gini',
 'max_depth': 25,
 'max_features': 3,
 'max_leaf_nodes': 16,
 'min_samples_leaf': 0.13,
 'min_samples_split': 0.13,
 'n_estimators': 53}

In [11]:
random_search.best_score_

0.1499401286603578

In [ ]:
RF = RandomForestClassifier(class_weight=class_weight,
                            n_estimators = 200,
                            criterion = 'gini',
                            max_features = 1,
                            max_depth = 10, 
                            min_samples_split = 0.15,
                            max_leaf_nodes = 200)

In [ ]:
# scores = cross_val_score(estimator=RF, X=X, y=y, cv=5, n_jobs=-1, scoring=gini_scorer)
# print('CV accuracy scores: %s' % scores); print()
# print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))